In [1]:
import findspark
findspark.init()

In [2]:
import pyspark
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark.sql.window import *

In [3]:
spark=SparkSession.builder.appName('practice').getOrCreate()
df=spark.sql("select 'spark' as hello")
df.show()

+-----+
|hello|
+-----+
|spark|
+-----+



In [5]:
activity=[(1,2,'2016-03-01',5),(1,2,'2016-05-02',2),(2,3,'2017-06-25',1),
            (3,1,'2016-03-02',0),(3,4,'2018-07-03',5)]
activitycolumns=["player_id","device_id","event_date","games_played"]
activitydf=spark.createDataFrame(activity,activitycolumns)
activitydf.show()

+---------+---------+----------+------------+
|player_id|device_id|event_date|games_played|
+---------+---------+----------+------------+
|        1|        2|2016-03-01|           5|
|        1|        2|2016-05-02|           2|
|        2|        3|2017-06-25|           1|
|        3|        1|2016-03-02|           0|
|        3|        4|2018-07-03|           5|
+---------+---------+----------+------------+



In [6]:
activitydf.createOrReplaceTempView("activity")

In [10]:
spark.sql("""SELECT player_id, MIN(event_date) AS first_login
FROM Activity  GROUP BY player_id""").show()

+---------+-----------+
|player_id|first_login|
+---------+-----------+
|        1| 2016-03-01|
|        2| 2017-06-25|
|        3| 2016-03-02|
+---------+-----------+



In [9]:
orders=[(1,1),(2,2),(3,3),(4,3),(5,2),(6,3)]
ordercolumns=["order_number","customer_number"]
orderdf=spark.createDataFrame(orders,ordercolumns)
orderdf.show()

+------------+---------------+
|order_number|customer_number|
+------------+---------------+
|           1|              1|
|           2|              2|
|           3|              3|
|           4|              3|
|           5|              2|
|           6|              3|
+------------+---------------+



In [11]:
orderdf.createOrReplaceTempView("orders")

In [15]:
spark.sql("""SELECT customer_number
FROM
    orders
GROUP BY customer_number order by count(*) desc limit 1""").show()

+---------------+
|customer_number|
+---------------+
|              3|
+---------------+



In [16]:
course=[('A', 'Mathematics'),('B', 'English'),('C', 'Mathematics'),
('D', 'Biology'),('E', 'Mathematics'),('F', 'Computer'),('G', 'Mathematics'),('H', 'Mathematics'),
('I', 'Mathematics'),('J','Biology')]
coursecolumns=["student","class"]
coursedf=spark.createDataFrame(course,coursecolumns)
coursedf.show()

+-------+-----------+
|student|      class|
+-------+-----------+
|      A|Mathematics|
|      B|    English|
|      C|Mathematics|
|      D|    Biology|
|      E|Mathematics|
|      F|   Computer|
|      G|Mathematics|
|      H|Mathematics|
|      I|Mathematics|
|      J|    Biology|
+-------+-----------+



In [17]:
coursedf.createOrReplaceTempView("courses")

In [62]:
spark.sql("""select class,count(*) from courses group by class having count(distinct student)>=2""").show()

+-----------+--------+
|      class|count(1)|
+-----------+--------+
|Mathematics|       6|
|    Biology|       2|
+-----------+--------+



In [33]:
tree=[('1', None),('2', '1'),('3', '1'),('4', '2'),('5', '2')]
treecolumns=["id","parent_id"]
treedf=spark.createDataFrame(tree,treecolumns)
treedf.show()

+---+---------+
| id|parent_id|
+---+---------+
|  1|     null|
|  2|        1|
|  3|        1|
|  4|        2|
|  5|        2|
+---+---------+



In [34]:
treedf.createOrReplaceTempView("Tree")

In [87]:
spark.sql("""SELECT
    id AS Id,    
    CASE 1=1
        WHEN tree.id = (SELECT t.id FROM tree t WHERE t.parent_id IS NULL)         
        THEN 'Root'        
        WHEN tree.id = (SELECT t.parent_id FROM tree t)        
        THEN 'Inner'     
        ELSE 'Leaf'   
        END AS Type
FROM
    tree
ORDER BY Id""").show()

Py4JJavaError: An error occurred while calling o370.showString.
: java.lang.IllegalStateException: more than one row returned by a subquery used as an expression:
Subquery subquery#649, [id=#2019]
+- AdaptiveSparkPlan isFinalPlan=true
   +- == Final Plan ==
      *(1) Project [parent_id#654]
      +- *(1) Scan ExistingRDD[id#653,parent_id#654]
   +- == Initial Plan ==
      Project [parent_id#654]
      +- Scan ExistingRDD[id#653,parent_id#654]

	at org.apache.spark.sql.execution.ScalarSubquery.updateResult(subquery.scala:84)
	at org.apache.spark.sql.execution.SparkPlan.$anonfun$waitForSubqueries$1(SparkPlan.scala:262)
	at org.apache.spark.sql.execution.SparkPlan.$anonfun$waitForSubqueries$1$adapted(SparkPlan.scala:261)
	at scala.collection.mutable.ResizableArray.foreach(ResizableArray.scala:62)
	at scala.collection.mutable.ResizableArray.foreach$(ResizableArray.scala:55)
	at scala.collection.mutable.ArrayBuffer.foreach(ArrayBuffer.scala:49)
	at org.apache.spark.sql.execution.Sp

In [54]:
actor=[('1', '1', '0'),('1', '1', '1'),('1', '1', '2'),('1', '2', '3'),
('1', '2', '4'),('2', '1', '5'),('2', '1', '6')]
actorcolumns=["actor_id","director_id","timestamp"]
actordf=spark.createDataFrame(actor,actorcolumns)
actordf.show()

+--------+-----------+---------+
|actor_id|director_id|timestamp|
+--------+-----------+---------+
|       1|          1|        0|
|       1|          1|        1|
|       1|          1|        2|
|       1|          2|        3|
|       1|          2|        4|
|       2|          1|        5|
|       2|          1|        6|
+--------+-----------+---------+



In [55]:
actordf.createOrReplaceTempView("actor")

In [58]:
spark.sql("""select actor_id,director_id from
(select actor_id,director_id,COUNT(timestamp) AS nums
FROM Actor
GROUP BY actor_id, director_id
ORDER BY actor_id, director_id) tmp
WHERE nums >= 3""").show()

+--------+-----------+
|actor_id|director_id|
+--------+-----------+
|       1|          1|
+--------+-----------+



In [63]:
identity=[(1,'Noori','Doctor'),(2,'Nayeem','Model'),(3,'Azgar','business'),
							(4,'Suhail','Actor'),(5,'Saif','Actor')]
identitycolumns=["id","name","profession"]
identitydf=spark.createDataFrame(identity,identitycolumns)
identitydf.show()

+---+------+----------+
| id|  name|profession|
+---+------+----------+
|  1| Noori|    Doctor|
|  2|Nayeem|     Model|
|  3| Azgar|  business|
|  4|Suhail|     Actor|
|  5|  Saif|     Actor|
+---+------+----------+



In [64]:
identitydf.createOrReplaceTempView("identity")

In [68]:
spark.sql("""select concat(name, '(', substr(profession,1,1), ')') as fullname from identity""").show()

+---------+
| fullname|
+---------+
| Noori(D)|
|Nayeem(M)|
| Azgar(b)|
|Suhail(A)|
|  Saif(A)|
+---------+



In [72]:
subscription=[(1,'2022-01-01','2022-01-31'),(2,'2022-01-16','2022-01-26'),
								(3,'2022-01-28','2022-02-06'),(4,'2022-02-16','2022-02-26')]
subscriptioncolumns=["user_id","start_date","end_date"]
subscriptiondf=spark.createDataFrame(subscription,subscriptioncolumns)
subscriptiondf.show()

+-------+----------+----------+
|user_id|start_date|  end_date|
+-------+----------+----------+
|      1|2022-01-01|2022-01-31|
|      2|2022-01-16|2022-01-26|
|      3|2022-01-28|2022-02-06|
|      4|2022-02-16|2022-02-26|
+-------+----------+----------+



In [73]:
subcriptiondf.createOrReplaceTempView("subscription")

In [75]:
spark.sql("""select distinct s1.user_id,(
  case when s2.user_id is not null then 'true' else 'false' end) as overlap
  from subscription as s1 left join subscription s2
  on s1.user_id != s2.user_id
  and s1.start_date<=s2.end_date
and s1.end_date>=s2.start_date""").show()

+-------+-------+
|user_id|overlap|
+-------+-------+
|      1|   true|
|      2|   true|
|      3|   true|
|      4|  false|
+-------+-------+



In [76]:
user=[('1', '2018-01-01', 'Lenovo'), ('2', '2018-02-09', 'Samsung'), 
('3', '2018-01-19', 'LG'), ('4', '2018-05-21', 'HP')]
usercolumns=["user_id","join_date","favorite_brand"]
userdf=spark.createDataFrame(user,usercolumns)
userdf.show()

+-------+----------+--------------+
|user_id| join_date|favorite_brand|
+-------+----------+--------------+
|      1|2018-01-01|        Lenovo|
|      2|2018-02-09|       Samsung|
|      3|2018-01-19|            LG|
|      4|2018-05-21|            HP|
+-------+----------+--------------+



In [77]:
orders1=[('1', '2019-08-01', '4', '1', '2'),
('2', '2018-08-02', '2', '1', '3'),('3', '2019-08-03', '3', '2', '3'),
('4', '2018-08-04', '1', '4', '2'),('5', '2018-08-04', '1', '3', '4'),
('6', '2019-08-05', '2', '2', '4')]
ordercolumns=["order_id","order_date","item_id","buyer_id","seller_id"]
orderdf=spark.createDataFrame(orders1,ordercolumns)
orderdf.show()

+--------+----------+-------+--------+---------+
|order_id|order_date|item_id|buyer_id|seller_id|
+--------+----------+-------+--------+---------+
|       1|2019-08-01|      4|       1|        2|
|       2|2018-08-02|      2|       1|        3|
|       3|2019-08-03|      3|       2|        3|
|       4|2018-08-04|      1|       4|        2|
|       5|2018-08-04|      1|       3|        4|
|       6|2019-08-05|      2|       2|        4|
+--------+----------+-------+--------+---------+



In [78]:
items=[('1', 'Samsung'),('2', 'Lenovo'),('3', 'LG'),('4', 'HP')]
itemcolumns=["item_id","item_brand"]
itemdf=spark.createDataFrame(items,itemcolumns)
itemdf.show()

+-------+----------+
|item_id|item_brand|
+-------+----------+
|      1|   Samsung|
|      2|    Lenovo|
|      3|        LG|
|      4|        HP|
+-------+----------+



In [89]:
userdf.createOrReplaceTempView("users")
orderdf.createOrReplaceTempView("orders1")
itemdf.createOrReplaceTempView("items")

In [91]:
spark.sql("""SELECT     u.user_id AS buyer_id,
           u.join_date,
           IFNULL(c.order_count, 0) AS orders_in_2019
FROM       Users u
LEFT JOIN (SELECT   buyer_id,
                    COUNT(buyer_id) AS order_count
           FROM    (SELECT buyer_id
                    FROM   Orders1
                    WHERE  order_date >= '2019-01-01') o
           GROUP BY buyer_id) c
ON         u.user_id = c.buyer_id
ORDER BY   u.user_id ASC""").show()

+--------+----------+--------------+
|buyer_id| join_date|orders_in_2019|
+--------+----------+--------------+
|       1|2018-01-01|             1|
|       2|2018-02-09|             2|
|       3|2018-01-19|             0|
|       4|2018-05-21|             0|
+--------+----------+--------------+



In [92]:
department=[('1', '8000', 'Jan'),('2', '9000', 'Jan'),
							('3', '10000', 'Feb'),('1', '7000', 'Feb'),
							('1', '6000', 'Mar'),('2','6000','Mar')]
departmentcolumns=["id","revenue","month"]
departmentdf=spark.createDataFrame(department,departmentcolumns)
departmentdf.show()

+---+-------+-----+
| id|revenue|month|
+---+-------+-----+
|  1|   8000|  Jan|
|  2|   9000|  Jan|
|  3|  10000|  Feb|
|  1|   7000|  Feb|
|  1|   6000|  Mar|
|  2|   6000|  Mar|
+---+-------+-----+



In [93]:
departmentdf.createOrReplaceTempView("department")

In [95]:
spark.sql("""SELECT id, 
SUM(IF (month = "Jan", revenue, null)) AS Jan_Revenue, 
SUM(IF (month = "Feb", revenue, null)) AS Feb_Revenue, 
SUM(IF (month = "Mar", revenue, null)) AS Mar_Revenue, 
SUM(IF (month = "Apr", revenue, null)) AS Apr_Revenue,
 SUM(IF (month = "May", revenue, null)) AS May_Revenue, 
 SUM(IF (month = "Jun", revenue, null)) AS Jun_Revenue,
 SUM(IF (month = "Jul", revenue, null)) AS Jul_Revenue, 
 SUM(IF (month = "Aug", revenue, null)) AS Aug_Revenue, 
 SUM(IF (month = "Sep", revenue, null)) AS Sep_Revenue,
 SUM(IF (month = "Oct", revenue, null)) AS Oct_Revenue, 
 SUM(IF (month = "Nov", revenue, null)) AS Nov_Revenue, 
 SUM(IF (month = "Dec", revenue, null)) AS Dec_Revenue 
 FROM Department GROUP BY id""").show()

+---+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+
| id|Jan_Revenue|Feb_Revenue|Mar_Revenue|Apr_Revenue|May_Revenue|Jun_Revenue|Jul_Revenue|Aug_Revenue|Sep_Revenue|Oct_Revenue|Nov_Revenue|Dec_Revenue|
+---+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+
|  1|     8000.0|     7000.0|     6000.0|       null|       null|       null|       null|       null|       null|       null|       null|       null|
|  2|     9000.0|       null|     6000.0|       null|       null|       null|       null|       null|       null|       null|       null|       null|
|  3|       null|    10000.0|       null|       null|       null|       null|       null|       null|       null|       null|       null|       null|
+---+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------

In [110]:
from enum import Enum
stock=[('Maggie', 'Buy', '1', '1000'),('Corona Masks', 'Buy', '2', '10'),
('Maggie', 'Sell', '5', '9000'),('Handbags', 'Buy', '17', '30000'),
('Corona Masks', 'Sell', '3', '1010'),('Corona Masks', 'Buy', '4', '1000'),
('Corona Masks', 'Sell', '5', '500'),('Corona Masks', 'Buy', '6', '1000'),
('Handbags', 'Sell', '29', '7000'),('Corona Masks', 'Sell', '10', '10000')]
operation=Enum('operation',['Buy','sell'])
stockcolumns=["stock_name","operation","operation_day","price"]
stockdf=spark.createDataFrame(stock,stockcolumns)
stockdf.show()

+------------+---------+-------------+-----+
|  stock_name|operation|operation_day|price|
+------------+---------+-------------+-----+
|      Maggie|      Buy|            1| 1000|
|Corona Masks|      Buy|            2|   10|
|      Maggie|     Sell|            5| 9000|
|    Handbags|      Buy|           17|30000|
|Corona Masks|     Sell|            3| 1010|
|Corona Masks|      Buy|            4| 1000|
|Corona Masks|     Sell|            5|  500|
|Corona Masks|      Buy|            6| 1000|
|    Handbags|     Sell|           29| 7000|
|Corona Masks|     Sell|           10|10000|
+------------+---------+-------------+-----+



In [111]:
stockdf.createOrReplaceTempView("stocks")

In [112]:
spark.sql("""SELECT stock_name, 
SUM( Case When operation='Buy' then -price When operation='Sell' then price End)
 As capital_gain_loss FROM Stocks Group By stock_name""").show()

+------------+-----------------+
|  stock_name|capital_gain_loss|
+------------+-----------------+
|      Maggie|           8000.0|
|Corona Masks|           9500.0|
|    Handbags|         -23000.0|
+------------+-----------------+

